# Tutorial 1: The Guild's Problem

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/blockchain-tokens-and-incentives/blob/main/notebooks/tutorial_01_the_guilds_problem.ipynb)

---

> *"The Capital tried rules. The Guild tried honor. Both failed. I tried mathematics — and discovered that honesty is just the cheapest strategy."*
>
> — Brenn Auster, *On the Alignment of Interest and Obligation* (Year 875)

---

## When Enforcers Defect

The Traders Guild operated across Densworld for centuries, facilitating commerce between regions with vastly different conditions. In stable territories near the Capital, guild rules worked: inspectors verified goods, penalties deterred fraud, and disputes were resolved through established procedures.

Then the Guild expanded into the Western Reaches and Threshold — territories near the Dens, where conditions were volatile. Prices shifted without explanation. Goods transmuted in transit. Contracts became ambiguous when the very properties they described changed.

The Capital's response was more enforcement: more inspectors, harsher penalties, mandatory documentation. But enforcement requires enforcers who are honest. Near the Dens, inspectors could be bribed. Reports vanished. Penalties were applied inconsistently — or not at all.

**Brenn Auster**, a guild economist, noticed the deeper problem: the enforcement system itself was a game where defection could be profitable. Inspectors who took bribes earned more than honest ones. Traders who cheated and bribed the inspector paid less than traders who played fair. The system *incentivized* the behavior it was designed to prevent.

Auster proposed a radical alternative: **design the rules so that honesty is the cheapest strategy.** Not because enforcers are watching — because the mathematics make cheating unprofitable regardless.

## Learning Objectives

In this tutorial, you will:

1. **Load the guild trade ledger** — 600 transactions across 20 years of guild commerce
2. **Measure enforcement failure** — fraud rates, inspector bribery, Dens proximity effects
3. **Model the Prisoner's Dilemma** — why rational traders defect even when cooperation pays more
4. **Quantify the enforcement gap** — where rules fail, and why more rules won't help
5. **Motivate mechanism design** — the idea that you can engineer incentive structures

**Technical Concepts:**
- The Prisoner's Dilemma as a model of trade relationships
- Payoff matrices and dominant strategies
- Why enforcement-based systems fail in adversarial environments
- The concept of incentive compatibility

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

guild_trades = pd.read_csv(BASE_URL + "guild_trade_ledger.csv")
scholars = pd.read_csv(BASE_URL + "scholars.csv")

print(f"Guild trade ledger: {len(guild_trades)} transactions")
print(f"Scholars registry: {len(scholars)} known scholars")
print(f"\nYears covered: {guild_trades['year'].min()}-{guild_trades['year'].max()}")
print(f"Unique traders: {guild_trades['seller'].nunique() + guild_trades['buyer'].nunique()}")
print(f"Commodities traded: {guild_trades['commodity'].nunique()}")

## The Trade Ledger

The guild's trade ledger spans Years 860-880 — two decades during which Auster developed his incentive framework. Each entry records a transaction: who sold, who bought, what commodity, at what price, on which route, and — critically — the outcome.

In [ ]:
print("Trade outcome distribution:")
outcome_counts = guild_trades['outcome'].value_counts()
print(outcome_counts)
print(f"\nCompletion rate: {(guild_trades['outcome'] == 'completed').mean():.1%}")
print(f"Dispute rate: {(guild_trades['outcome'] == 'disputed').mean():.1%}")
print(f"Default rate: {(guild_trades['outcome'] == 'defaulted').mean():.1%}")
print(f"\nFraud detected in: {guild_trades['fraud_detected'].sum()} trades ({guild_trades['fraud_detected'].mean():.1%})")

In [ ]:
# Outcome breakdown by trade route
route_outcomes = guild_trades.groupby('trade_route')['outcome'].value_counts().unstack(fill_value=0)
route_outcomes['total'] = route_outcomes.sum(axis=1)
route_outcomes['failure_rate'] = 1 - route_outcomes.get('completed', 0) / route_outcomes['total']
route_outcomes = route_outcomes.sort_values('failure_rate', ascending=False)

print("Failure rate by trade route:")
for route, row in route_outcomes.iterrows():
    print(f"  {route:25s} {row['failure_rate']:.1%} failure ({int(row['total'])} trades)")

## The Dens Proximity Effect

> *"The Capital's rules assume stable ground. Near the Dens, the ground itself is the problem."*
>
> — Brenn Auster, *Self-Enforcing Agreements* (Year 867)

Auster's first insight was that fraud correlated with proximity to the Dens. Let's visualize this.

In [ ]:
# Fraud rate vs. Dens proximity
proximity_fraud = guild_trades.groupby('dens_proximity').agg(
    total_trades=('trade_id', 'count'),
    fraud_count=('fraud_detected', 'sum'),
    disputed=('outcome', lambda x: (x == 'disputed').sum()),
    defaulted=('outcome', lambda x: (x == 'defaulted').sum())
).reset_index()
proximity_fraud['fraud_rate'] = proximity_fraud['fraud_count'] / proximity_fraud['total_trades']
proximity_fraud['problem_rate'] = (
    (proximity_fraud['disputed'] + proximity_fraud['defaulted']) / proximity_fraud['total_trades']
)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.bar(proximity_fraud['dens_proximity'], proximity_fraud['fraud_rate'],
        color='firebrick', alpha=0.8, edgecolor='black')
ax1.set_xlabel('Dens Proximity (0=far, 1=near)')
ax1.set_ylabel('Fraud Detection Rate')
ax1.set_title('Fraud Rate vs. Dens Proximity')

ax2.bar(proximity_fraud['dens_proximity'], proximity_fraud['problem_rate'],
        color='darkorange', alpha=0.8, edgecolor='black')
ax2.set_xlabel('Dens Proximity (0=far, 1=near)')
ax2.set_ylabel('Dispute + Default Rate')
ax2.set_title('Problem Rate vs. Dens Proximity')

plt.tight_layout()
plt.show()

## Inspector Failure

The Capital's solution to trade fraud was inspectors — guild officials who would verify goods, mediate disputes, and apply penalties. But inspectors are people, and people respond to incentives.

In [ ]:
# Inspector involvement and rulings
inspected = guild_trades[guild_trades['inspector_involved'] == True].copy()
print(f"Trades with inspector involvement: {len(inspected)}/{len(guild_trades)} ({len(inspected)/len(guild_trades):.1%})")

ruling_counts = inspected[inspected['inspector_ruling'] != '']['inspector_ruling'].value_counts()
print(f"\nInspector rulings (where applicable):")
for ruling, count in ruling_counts.items():
    print(f"  {ruling:25s} {count:3d}")

bribed = (inspected['inspector_ruling'] == 'inspector_bribed').sum()
total_rulings = (inspected['inspector_ruling'] != '').sum()
print(f"\nBribery rate among rulings: {bribed}/{total_rulings} ({bribed/max(total_rulings,1):.1%})")

In [ ]:
# Who are the inspectors? (Top 10 guild members assigned as inspectors)
inspector_stats = inspected[inspected['inspector'] != ''].groupby('inspector').agg(
    cases=('trade_id', 'count'),
    bribed=('inspector_ruling', lambda x: (x == 'inspector_bribed').sum())
).reset_index()
inspector_stats['bribe_rate'] = inspector_stats['bribed'] / inspector_stats['cases']
inspector_stats = inspector_stats.sort_values('cases', ascending=False)

print("Inspector performance:")
for _, row in inspector_stats.iterrows():
    flag = " *** COMPROMISED" if row['bribe_rate'] > 0.15 else ""
    print(f"  {row['inspector']:20s} {int(row['cases']):3d} cases, "
          f"{int(row['bribed']):2d} bribes ({row['bribe_rate']:.0%}){flag}")

## The Prisoner's Dilemma

Auster recognized the guild's problem as a **Prisoner's Dilemma**: two traders can each choose to cooperate (trade honestly) or defect (cheat). The payoffs create a trap.

| | Trader B Cooperates | Trader B Defects |
|---|---|---|
| **Trader A Cooperates** | Both earn 3 (mutual gain) | A gets 0, B gets 5 (sucker's payoff) |
| **Trader A Defects** | A gets 5, B gets 0 (temptation) | Both earn 1 (mutual punishment) |

The rational choice for each trader — regardless of what the other does — is to **defect**. This is the **dominant strategy**. But if both defect, they each earn 1 instead of the 3 they'd get from mutual cooperation.

In [ ]:
# Implement the one-shot Prisoner's Dilemma
def prisoners_dilemma(action_a, action_b):
    """Returns (payoff_a, payoff_b) for a single round."""
    payoffs = {
        ('cooperate', 'cooperate'): (3, 3),  # R: Reward for mutual cooperation
        ('cooperate', 'defect'):    (0, 5),  # S,T: Sucker's payoff, Temptation
        ('defect', 'cooperate'):    (5, 0),  # T,S: Temptation, Sucker's payoff
        ('defect', 'defect'):       (1, 1),  # P: Punishment for mutual defection
    }
    return payoffs[(action_a, action_b)]

# Show the payoff matrix
print("Payoff Matrix (A's payoff, B's payoff):")
print(f"{'':15s} {'B Cooperates':>15s} {'B Defects':>15s}")
print(f"{'A Cooperates':15s} {str(prisoners_dilemma('cooperate','cooperate')):>15s} "
      f"{str(prisoners_dilemma('cooperate','defect')):>15s}")
print(f"{'A Defects':15s} {str(prisoners_dilemma('defect','cooperate')):>15s} "
      f"{str(prisoners_dilemma('defect','defect')):>15s}")

print("\nDominant strategy analysis:")
print("  If B cooperates: A gets 3 (cooperate) vs 5 (defect) → defect wins")
print("  If B defects:    A gets 0 (cooperate) vs 1 (defect) → defect wins")
print("  Defect dominates regardless. Both defect. Both earn 1.")
print("  But mutual cooperation would give both 3. This is the dilemma.")

In [ ]:
# Simulate 1000 one-shot Prisoner's Dilemma games
# Population: 70% cooperators, 30% defectors (like the guild trade data)
np.random.seed(867)  # Year of Auster's first paper
n_games = 1000

results = []
for _ in range(n_games):
    a = 'cooperate' if np.random.random() < 0.7 else 'defect'
    b = 'cooperate' if np.random.random() < 0.7 else 'defect'
    pa, pb = prisoners_dilemma(a, b)
    results.append({'action_a': a, 'action_b': b, 'payoff_a': pa, 'payoff_b': pb})

sim_df = pd.DataFrame(results)

# Average payoffs by strategy
print("Average payoffs in 1000 one-shot games (70% cooperators):")
for action in ['cooperate', 'defect']:
    avg = sim_df[sim_df['action_a'] == action]['payoff_a'].mean()
    count = (sim_df['action_a'] == action).sum()
    print(f"  {action:10s}: avg payoff = {avg:.2f} (played {count} times)")

print(f"\nDefectors earn more on average in one-shot games.")
print(f"This is why enforcement fails: cheating is rational.")

## Mapping Guild Trades to the Dilemma

The guild trade data maps directly to the Prisoner's Dilemma. A "completed" trade with no dispute is mutual cooperation. A "disputed" or "defaulted" trade involves at least one defector.

In [ ]:
# Map trade outcomes to PD categories
def classify_trade(row):
    if row['outcome'] == 'completed' and row['dispute_type'] == 'none':
        return 'mutual_cooperation'
    elif row['outcome'] == 'completed' and row['dispute_type'] != 'none':
        return 'cooperation_with_friction'
    elif row['outcome'] == 'disputed':
        return 'at_least_one_defected'
    else:  # defaulted
        return 'clear_defection'

guild_trades['pd_category'] = guild_trades.apply(classify_trade, axis=1)

print("Guild trades mapped to Prisoner's Dilemma:")
cat_counts = guild_trades['pd_category'].value_counts()
for cat, count in cat_counts.items():
    print(f"  {cat:30s} {count:4d} ({count/len(guild_trades):.1%})")

print(f"\nTotal non-cooperative trades: "
      f"{(guild_trades['pd_category'].isin(['at_least_one_defected','clear_defection'])).sum()}")
print(f"This is the guild's enforcement gap.")

In [ ]:
# Visualize the enforcement gap over time
yearly = guild_trades.groupby('year').agg(
    total=('trade_id', 'count'),
    defections=('pd_category', lambda x: x.isin(['at_least_one_defected', 'clear_defection']).sum()),
    fraud=('fraud_detected', 'sum')
).reset_index()
yearly['defection_rate'] = yearly['defections'] / yearly['total']
yearly['fraud_rate'] = yearly['fraud'] / yearly['total']

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(yearly['year'], yearly['defection_rate'], 'o-', color='firebrick',
        label='Defection rate (disputes + defaults)', linewidth=2)
ax.plot(yearly['year'], yearly['fraud_rate'], 's--', color='darkorange',
        label='Detected fraud rate', linewidth=2)
ax.set_xlabel('Year')
ax.set_ylabel('Rate')
ax.set_title('Guild Trade Defection Over Time (Years 860-880)')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## The Enforcement Paradox

Auster identified a paradox: **more enforcement can make things worse.** Adding inspectors raises the cost of doing business for honest traders (they pay inspection fees) while creating new opportunities for corruption (inspectors can be bribed).

In [ ]:
# Cost of enforcement: trades with inspectors vs. without
inspected_trades = guild_trades[guild_trades['inspector_involved'] == True]
uninspected_trades = guild_trades[guild_trades['inspector_involved'] == False]

print("INSPECTED TRADES:")
print(f"  Total: {len(inspected_trades)}")
print(f"  Fraud detected: {inspected_trades['fraud_detected'].sum()} "
      f"({inspected_trades['fraud_detected'].mean():.1%})")
print(f"  Disputed: {(inspected_trades['outcome']=='disputed').sum()} "
      f"({(inspected_trades['outcome']=='disputed').mean():.1%})")
bribed_count = (inspected_trades['inspector_ruling'] == 'inspector_bribed').sum()
print(f"  Inspector bribed: {bribed_count}")

print(f"\nUNINSPECTED TRADES:")
print(f"  Total: {len(uninspected_trades)}")
print(f"  Fraud detected: {uninspected_trades['fraud_detected'].sum()} "
      f"({uninspected_trades['fraud_detected'].mean():.1%})")
print(f"  Disputed: {(uninspected_trades['outcome']=='disputed').sum()} "
      f"({(uninspected_trades['outcome']=='disputed').mean():.1%})")

print(f"\nThe paradox: inspected trades show higher dispute rates")
print(f"because inspectors are assigned to problematic trades.")
print(f"But {bribed_count} bribed inspectors means enforcement itself is compromised.")

In [ ]:
# Model the enforcement cost
# Honest trade: commodity value + inspection fee (5%)
# Dishonest trade (undetected): commodity value - stolen margin (~20%)
# Dishonest trade (detected): commodity value + penalty (~30%)
# Dishonest trade (bribed inspector): commodity value - bribe (~10%) + stolen margin

inspection_fee = 0.05
stolen_margin = 0.20
penalty_rate = 0.30
bribe_cost = 0.10
detection_prob = 0.40  # Only 40% of fraud is detected

avg_value = guild_trades['total_value'].mean()

honest_payoff = avg_value * (1 - inspection_fee)
cheat_undetected = avg_value * (1 + stolen_margin)
cheat_detected = avg_value * (1 - penalty_rate)
cheat_bribed = avg_value * (1 + stolen_margin - bribe_cost)

# Expected payoff of cheating (with enforcement)
bribe_rate = bribed_count / max(len(inspected_trades), 1)
expected_cheat = (
    (1 - detection_prob) * cheat_undetected +
    detection_prob * (1 - bribe_rate) * cheat_detected +
    detection_prob * bribe_rate * cheat_bribed
)

print(f"Average trade value: {avg_value:.2f}")
print(f"\nExpected payoffs:")
print(f"  Honest trading:  {honest_payoff:.2f}")
print(f"  Cheating:        {expected_cheat:.2f}")
print(f"  Difference:      {expected_cheat - honest_payoff:+.2f}")
print(f"\nCheating is {'more' if expected_cheat > honest_payoff else 'less'} "
      f"profitable than honesty by {abs(expected_cheat - honest_payoff):.2f} per trade.")
print(f"This is Auster's core insight: the system rewards defection.")

## The Commodity Landscape

Before proposing a solution, Auster mapped the full landscape of guild commerce — which commodities were traded, on which routes, with what risks.

In [ ]:
# Commodity analysis
commodity_stats = guild_trades.groupby('commodity').agg(
    trades=('trade_id', 'count'),
    avg_price=('agreed_price', 'mean'),
    total_volume=('total_value', 'sum'),
    fraud_rate=('fraud_detected', 'mean'),
    dispute_rate=('outcome', lambda x: (x != 'completed').mean())
).round(2).sort_values('total_volume', ascending=False)

print("Guild commerce by commodity:")
print(f"{'Commodity':20s} {'Trades':>6s} {'Avg Price':>10s} {'Volume':>12s} "
      f"{'Fraud%':>7s} {'Dispute%':>9s}")
for commodity, row in commodity_stats.iterrows():
    print(f"{commodity:20s} {int(row['trades']):6d} {row['avg_price']:10.2f} "
          f"{row['total_volume']:12.2f} {row['fraud_rate']:7.1%} {row['dispute_rate']:9.1%}")

In [ ]:
# Top 8 commodities by volume
top8 = commodity_stats.head(8)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.barh(top8.index, top8['total_volume'], color='steelblue', alpha=0.8, edgecolor='black')
ax1.set_xlabel('Total Trade Volume')
ax1.set_title('Top 8 Commodities by Volume')

colors = ['firebrick' if r > 0.10 else 'steelblue' for r in top8['fraud_rate']]
ax2.barh(top8.index, top8['fraud_rate'], color=colors, alpha=0.8, edgecolor='black')
ax2.set_xlabel('Fraud Rate')
ax2.set_title('Fraud Rate by Commodity (red = >10%)')

plt.tight_layout()
plt.show()

## The Game Theory Preview

The guild trade data has a companion dataset: `game_theory_outcomes.csv`, which records 30 matches of the iterated Prisoner's Dilemma between guild members. Let's preview it.

In [ ]:
game_theory = pd.read_csv(BASE_URL + "game_theory_outcomes.csv")
print(f"Game theory outcomes: {len(game_theory)} rounds across "
      f"{game_theory['match_id'].nunique()} matches")

# Which strategies appear?
print(f"\nStrategies in the tournament:")
all_strategies = set(game_theory['strategy_a'].unique()) | set(game_theory['strategy_b'].unique())
for s in sorted(all_strategies):
    count = ((game_theory['strategy_a'] == s) | (game_theory['strategy_b'] == s)).sum()
    print(f"  {s:30s} appeared in {count:3d} rounds")

In [ ]:
# Which strategy wins? Average final-round cumulative payoff
final_rounds = game_theory[game_theory['round'] == 10]

strategy_performance_a = final_rounds.groupby('strategy_a')['cumulative_payoff_a'].mean()
strategy_performance_b = final_rounds.groupby('strategy_b')['cumulative_payoff_b'].mean()

# Combine both sides
all_perf = pd.concat([
    strategy_performance_a.rename('avg_payoff'),
    strategy_performance_b.rename('avg_payoff')
])
strategy_avg = all_perf.groupby(level=0).mean().sort_values(ascending=False)

print("Strategy rankings (average cumulative payoff over 10 rounds):")
for strategy, avg in strategy_avg.items():
    bar = '█' * int(avg)
    print(f"  {strategy:30s} {avg:5.1f}  {bar}")

print(f"\nIn repeated games, cooperative strategies tend to outperform")
print(f"pure defection. This is Auster's opening: repetition changes the game.")

## Auster's Insight: From Rules to Incentives

> *"Do not ask whether the guild member will cheat. Ask what it will cost them. Set the cost higher than the gain. Then it doesn't matter what they would do — only what they will do."*
>
> — Brenn Auster

The Prisoner's Dilemma shows that **one-shot interactions** drive defection. But guild members trade repeatedly. In repeated games, strategies like **tit-for-tat** — cooperate first, then mirror what your partner did — can sustain cooperation.

Auster's breakthrough was asking: **can we design the game itself?** Instead of policing behavior after the fact, engineer the rules so that cooperation is the dominant strategy. This is **mechanism design** — the reverse of game theory.

In [ ]:
# The staking preview: what if traders had to post collateral?
# If you cheat, you lose your stake. If you're honest, you get it back + reward.
stake_amount = avg_value * 0.5  # 50% of trade value as collateral
reward_rate = 0.05  # 5% return for honest trading

honest_with_stake = avg_value * (1 - inspection_fee) + stake_amount * reward_rate
cheat_with_stake = cheat_undetected - stake_amount  # Lose entire stake if caught

expected_cheat_with_stake = (
    (1 - detection_prob) * (cheat_undetected + stake_amount * reward_rate) +
    detection_prob * (cheat_detected - stake_amount)
)

print(f"WITHOUT STAKING (current system):")
print(f"  Honest payoff:   {honest_payoff:.2f}")
print(f"  Expected cheat:  {expected_cheat:.2f}")
print(f"  Cheating advantage: {expected_cheat - honest_payoff:+.2f}")

print(f"\nWITH STAKING (Auster's proposal):")
print(f"  Stake required:  {stake_amount:.2f} (50% of avg trade value)")
print(f"  Honest payoff:   {honest_with_stake:.2f}")
print(f"  Expected cheat:  {expected_cheat_with_stake:.2f}")
print(f"  Cheating advantage: {expected_cheat_with_stake - honest_with_stake:+.2f}")

if expected_cheat_with_stake < honest_with_stake:
    print(f"\n→ With staking, honesty becomes the dominant strategy.")
    print(f"  No inspectors needed. The math enforces itself.")
else:
    print(f"\n→ Stake needs to be higher to flip the incentive.")

## The Course Ahead

Auster's journey from observation to implementation required building several layers of economic and computational infrastructure:

| Tutorial | Title | What We Build |
|----------|-------|---------------|
| **01** | **The Guild's Problem** | **(This tutorial)** — The enforcement failure and Prisoner's Dilemma |
| 02 | Game Theory Foundations | Nash equilibrium, dominant strategies, repeated games, tit-for-tat |
| 03 | Mechanism Design | Reverse game theory: design games with desired equilibria |
| 04 | Token Standards (ERC-20, ERC-721) | What tokens are — state entries in a smart contract |
| 05 | Staking and Slashing | Lock up tokens to participate, lose them for misbehavior |
| 06 | Liquidity and AMMs | Automated market makers: trustless commodity exchange |
| 07 | MEV and Front-Running | When validators extract value from transaction ordering |
| 08 | Capstone: The Guild Economy | Complete token economy with staking, AMMs, and agent simulation |

## Exercises

### Exercise 1: The Worst Trade Route

Find the trade route with the highest combined rate of disputes, defaults, and fraud. What is the Dens proximity of that route? Does the data support Auster's claim that enforcement fails near the Dens?

In [ ]:
route_analysis = guild_trades.groupby('trade_route').agg(
    trades=('trade_id', 'count'),
    problem_rate=('pd_category', lambda x: x.isin(['at_least_one_defected', 'clear_defection']).mean()),
    fraud_rate=('fraud_detected', 'mean'),
    avg_proximity=('dens_proximity', 'mean')
).sort_values('problem_rate', ascending=False)

print("Trade routes ranked by problem rate:")
for route, row in route_analysis.head(5).iterrows():
    print(f"  {route:25s} problem={row['problem_rate']:.1%} "
          f"fraud={row['fraud_rate']:.1%} dens_proximity={row['avg_proximity']:.1f}")

### Exercise 2: The Honest Traders

Some guild members never appear in fraud cases — either as perpetrators or victims. Who are the consistently honest traders? Do they trade on specific routes or with specific commodities?

In [ ]:
# Find traders with zero fraud involvement
fraud_trades = guild_trades[guild_trades['fraud_detected'] == True]
fraud_participants = set(fraud_trades['seller'].unique()) | set(fraud_trades['buyer'].unique())
all_participants = set(guild_trades['seller'].unique()) | set(guild_trades['buyer'].unique())
honest_traders = all_participants - fraud_participants

print(f"Traders with zero fraud involvement: {len(honest_traders)}/{len(all_participants)}")
for trader in sorted(honest_traders):
    trader_trades = guild_trades[
        (guild_trades['seller'] == trader) | (guild_trades['buyer'] == trader)
    ]
    routes = trader_trades['trade_route'].value_counts().head(2)
    print(f"  {trader:20s} {len(trader_trades):3d} trades, "
          f"top routes: {', '.join(routes.index[:2])}")

### Exercise 3: Staking Threshold

What minimum stake (as a percentage of trade value) makes honesty the dominant strategy? Vary the stake from 10% to 100% and plot the expected payoff difference between cheating and honesty.

In [ ]:
stake_pcts = np.linspace(0.1, 1.0, 20)
advantages = []

for pct in stake_pcts:
    stake = avg_value * pct
    honest_p = avg_value * (1 - inspection_fee) + stake * reward_rate
    cheat_p = (
        (1 - detection_prob) * (cheat_undetected + stake * reward_rate) +
        detection_prob * (cheat_detected - stake)
    )
    advantages.append(cheat_p - honest_p)

plt.figure(figsize=(10, 5))
plt.plot(stake_pcts * 100, advantages, 'o-', color='firebrick', linewidth=2)
plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
plt.fill_between(stake_pcts * 100, advantages, 0,
                 where=[a > 0 for a in advantages], color='firebrick', alpha=0.1, label='Cheating profitable')
plt.fill_between(stake_pcts * 100, advantages, 0,
                 where=[a <= 0 for a in advantages], color='steelblue', alpha=0.1, label='Honesty profitable')
plt.xlabel('Stake (% of Trade Value)')
plt.ylabel('Cheating Advantage (positive = cheating pays)')
plt.title('Finding the Incentive Flip Point')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Find crossover
for i, (pct, adv) in enumerate(zip(stake_pcts, advantages)):
    if adv <= 0:
        print(f"Honesty becomes dominant at ~{pct*100:.0f}% stake")
        break

## Summary

In this tutorial, we learned:

1. **The guild trade ledger** reveals systematic enforcement failure — 30% of trades involve disputes or defaults
2. **Fraud correlates with Dens proximity** — enforcement fails where conditions are most volatile
3. **Inspectors can be bribed** — enforcement creates new vectors for corruption
4. **The Prisoner's Dilemma** explains why: defection is the dominant strategy in one-shot games
5. **Staking flips the incentive** — posting collateral makes honesty the rational choice

**Key insight:** Enforcement-based systems require honest enforcers. Incentive-based systems require only rational actors. Auster's contribution was recognizing that you don't need to make people good — you need to make goodness cheap.

---

**Next Tutorial:** Game Theory Foundations — Nash equilibrium, dominant strategies, and why tit-for-tat changes everything.

---

> *"Do not ask whether the guild member will cheat. Ask what it will cost them. Set the cost higher than the gain. Then it doesn't matter what they would do — only what they will do."*
>
> — Brenn Auster